## 7324 Assignment A6 : NN and PCA
##### Name: Thang Nguyen
##### SMU ID: 48689334

## Imports

In [1]:
# The Classifiers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.decomposition import PCA
# Data Wrangling Tool
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Foundational libraries
import pandas as pd
import tensorflow as tf
import keras
import numpy as np

## Loading Dataset

In [2]:
# note: I added the column names manually into the csv prior
# original csv from UCI comes without columns
abalone_df = pd.read_csv("../data/abalone.csv")
# checking dataframe
abalone_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sex             4177 non-null   object 
 1   Length          4177 non-null   float64
 2   Diameter        4177 non-null   float64
 3   Height          4177 non-null   float64
 4   Whole Weight    4177 non-null   float64
 5   Shucked Weight  4177 non-null   float64
 6   Viscera Weight  4177 non-null   float64
 7   Shell Weight    4177 non-null   float64
 8   Rings           4177 non-null   int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 293.8+ KB


In [3]:
abalone_df.head()

,Sex,Length,Diameter,Height,Whole Weight,Shucked Weight,Viscera Weight,Shell Weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


## Cleaning Dataset

### One-hot Encoding abalone sex

In [4]:
abalone_df = pd.concat([abalone_df, pd.get_dummies(abalone_df['Sex'], prefix = 'Sex')], axis = 1)
abalone_df.drop(['Sex'], axis = 1, inplace = True)

In [5]:
abalone_df.head()

,Length,Diameter,Height,Whole Weight,Shucked Weight,Viscera Weight,Shell Weight,Rings,Sex_F,Sex_I,Sex_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,0,0,1
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,0,0,1
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,1,0,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,0,0,1
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,0,1,0


### Categorizing Rings into groups where
#### 0-4 -> infant
#### 5-9 -> young
#### 10-14 -> teen
#### 15-20 -> young adult
#### 20-25 -> adult

In [6]:
def group_rings(rings):
    if rings <= 4:
        return 0
    if rings <= 9:
        return 1
    if rings <= 14:
        return 2
    if rings <= 20: 
        return 3
    if rings <= 25:
        return 4

In [7]:
abalone_df["Rings"] = abalone_df["Rings"].apply(group_rings)

In [8]:
abalone_df.head()

,Length,Diameter,Height,Whole Weight,Shucked Weight,Viscera Weight,Shell Weight,Rings,Sex_F,Sex_I,Sex_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,3.0,0,0,1
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,1.0,0,0,1
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,1.0,1,0,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,2.0,0,0,1
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,1.0,0,1,0


## Analysis

### Split and Scale

In [9]:
target = abalone_df["Rings"].values
features = abalone_df.drop('Rings', axis=1)

In [10]:
# scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, target, random_state = 0, train_size = 0.8)

In [12]:
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.int32)

## Building Keras NN

### 1 input/output Layer

#### inputs: 10 features
#### output layer: 5 target groups

In [13]:
model = Sequential()

In [14]:
model.add(Dense(3, input_dim = X_train.shape[1], activation='relu'))
model.add(Dense(5, activation='softmax'))

In [15]:
# must use categorical and spars
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [16]:
model.fit(X_train, y_train, epochs=500)

Epoch 1/500
  1/105 [..............................] - ETA: 18s - loss: 2.6072 - accuracy: 0.0000e+00

2023-04-15 17:01:30.330096: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


105/105 [==============================] - 0s 425us/step - loss: 1.6952 - accuracy: 0.2239
Epoch 2/500
105/105 [==============================] - 0s 350us/step - loss: 1.3038 - accuracy: 0.5606
Epoch 3/500
105/105 [==============================] - 0s 341us/step - loss: 1.1629 - accuracy: 0.6235
Epoch 4/500
105/105 [==============================] - 0s 504us/step - loss: 1.0898 - accuracy: 0.6351
Epoch 5/500
105/105 [==============================] - 0s 340us/step - loss: 1.0462 - accuracy: 0.6348
Epoch 6/500
105/105 [==============================] - 0s 339us/step - loss: 1.0169 - accuracy: 0.6384
Epoch 7/500
105/105 [==============================] - 0s 336us/step - loss: 0.9958 - accuracy: 0.6390
Epoch 8/500
105/105 [==============================] - 0s 333us/step - loss: 0.9796 - accuracy: 0.6411
Epoch 9/500
105/105 [==============================] - 0s 329us/step - loss: 0.9668 - accuracy: 0.6411
Epoch 10/500
105/105 [==============================] - 0s 332us/step - loss: 0.9563 

### 2 inner Layer

In [17]:
model = Sequential()

In [18]:
model.add(Dense(3, input_dim = X_train.shape[1], activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [19]:
# must use categorical and spars
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [20]:
model.fit(X_train, y_train, epochs=500)

Epoch 1/500
105/105 [==============================] - 0s 421us/step - loss: 1.5336 - accuracy: 0.3526
Epoch 2/500
105/105 [==============================] - 0s 343us/step - loss: 1.3714 - accuracy: 0.4762
Epoch 3/500
105/105 [==============================] - 0s 343us/step - loss: 1.2756 - accuracy: 0.4795
Epoch 4/500
105/105 [==============================] - 0s 342us/step - loss: 1.2128 - accuracy: 0.4795
Epoch 5/500
105/105 [==============================] - 0s 344us/step - loss: 1.1704 - accuracy: 0.4798
Epoch 6/500
105/105 [==============================] - 0s 345us/step - loss: 1.1405 - accuracy: 0.4798
Epoch 7/500
105/105 [==============================] - 0s 501us/step - loss: 1.1185 - accuracy: 0.4798
Epoch 8/500
105/105 [==============================] - 0s 354us/step - loss: 1.1019 - accuracy: 0.4798
Epoch 9/500
105/105 [==============================] - 0s 341us/step - loss: 1.0886 - accuracy: 0.4798
Epoch 10/500
105/105 [==============================] - 0s 350us/step - l

### 3 inner Layers

In [21]:
model = Sequential()

In [22]:
model.add(Dense(3, input_dim = X_train.shape[1], activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [23]:
# must use categorical and spars
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [24]:
model.fit(X_train, y_train, epochs=500)

Epoch 1/500
105/105 [==============================] - 0s 437us/step - loss: 1.4452 - accuracy: 0.4780
Epoch 2/500
105/105 [==============================] - 0s 365us/step - loss: 1.2165 - accuracy: 0.4801
Epoch 3/500
105/105 [==============================] - 0s 372us/step - loss: 1.1228 - accuracy: 0.5756
Epoch 4/500
105/105 [==============================] - 0s 372us/step - loss: 1.0631 - accuracy: 0.6336
Epoch 5/500
105/105 [==============================] - 0s 379us/step - loss: 1.0214 - accuracy: 0.6348
Epoch 6/500
105/105 [==============================] - 0s 371us/step - loss: 0.9911 - accuracy: 0.6375
Epoch 7/500
105/105 [==============================] - 0s 374us/step - loss: 0.9689 - accuracy: 0.6408
Epoch 8/500
105/105 [==============================] - 0s 371us/step - loss: 0.9504 - accuracy: 0.6417
Epoch 9/500
105/105 [==============================] - 0s 374us/step - loss: 0.9357 - accuracy: 0.6429
Epoch 10/500
105/105 [==============================] - 0s 367us/step - l

### Tuning: Increasing neurons using prime numbers => 
#### since feature set is relatively small but contains many observations,
#### increasing neurons aids in improving learning rate given 500 epochs

In [25]:
model = Sequential()

In [26]:
model.add(Dense(19, input_dim = X_train.shape[1], activation='relu'))
model.add(Dense(17, activation='relu'))
model.add(Dense(13, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [27]:
# must use categorical and spars
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [28]:
model.fit(X_train, y_train, epochs=500)

Epoch 1/500
105/105 [==============================] - 0s 561us/step - loss: 1.2993 - accuracy: 0.5172
Epoch 2/500
105/105 [==============================] - 0s 566us/step - loss: 0.9672 - accuracy: 0.6537
Epoch 3/500
105/105 [==============================] - 0s 572us/step - loss: 0.8789 - accuracy: 0.6513
Epoch 4/500
105/105 [==============================] - 0s 549us/step - loss: 0.8310 - accuracy: 0.6579
Epoch 5/500
105/105 [==============================] - 0s 570us/step - loss: 0.7904 - accuracy: 0.6627
Epoch 6/500
105/105 [==============================] - 0s 576us/step - loss: 0.7612 - accuracy: 0.6740
Epoch 7/500
105/105 [==============================] - 0s 733us/step - loss: 0.7420 - accuracy: 0.6866
Epoch 8/500
105/105 [==============================] - 0s 562us/step - loss: 0.7275 - accuracy: 0.6809
Epoch 9/500
105/105 [==============================] - 0s 533us/step - loss: 0.7149 - accuracy: 0.6920
Epoch 10/500
105/105 [==============================] - 0s 485us/step - l

### Tuning: Adjusting loss function with increased neurons =>
#### Nadam is Adam with Nesterov momentum [Keras API doc]

In [29]:
model = Sequential()

In [30]:
model.add(Dense(19, input_dim = X_train.shape[1], activation='relu'))
model.add(Dense(17, activation='relu'))
model.add(Dense(13, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [31]:
# must use categorical and sparse
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.Nadam(learning_rate=0.01),
              metrics=['accuracy'])

In [32]:
model.fit(X_train, y_train, epochs=500)

Epoch 1/500
105/105 [==============================] - 0s 550us/step - loss: 0.9113 - accuracy: 0.6315
Epoch 2/500
105/105 [==============================] - 0s 425us/step - loss: 0.7382 - accuracy: 0.6794
Epoch 3/500
105/105 [==============================] - 0s 443us/step - loss: 0.7101 - accuracy: 0.6803
Epoch 4/500
105/105 [==============================] - 0s 442us/step - loss: 0.6890 - accuracy: 0.7001
Epoch 5/500
105/105 [==============================] - 0s 439us/step - loss: 0.6895 - accuracy: 0.6896
Epoch 6/500
105/105 [==============================] - 0s 421us/step - loss: 0.6843 - accuracy: 0.7004
Epoch 7/500
105/105 [==============================] - 0s 425us/step - loss: 0.6801 - accuracy: 0.7043
Epoch 8/500
105/105 [==============================] - 0s 423us/step - loss: 0.6758 - accuracy: 0.7115
Epoch 9/500
105/105 [==============================] - 0s 424us/step - loss: 0.6721 - accuracy: 0.6995
Epoch 10/500
105/105 [==============================] - 0s 423us/step - l

### Tuning: Increasing Epochs with Nadam loss function and increased neurons =>
#### more epochs generally allows for more matured weights but learning can stagnate

In [33]:
model = Sequential()

In [34]:
model.add(Dense(19, input_dim = X_train.shape[1], activation='relu'))
model.add(Dense(17, activation='relu'))
model.add(Dense(13, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [35]:
# must use categorical and sparse
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.Nadam(learning_rate=0.01),
              metrics=['accuracy'])

In [36]:
model.fit(X_train, y_train, epochs=1000)

Epoch 1/1000
105/105 [==============================] - 0s 560us/step - loss: 0.8574 - accuracy: 0.6516
Epoch 2/1000
105/105 [==============================] - 0s 445us/step - loss: 0.7288 - accuracy: 0.6827
Epoch 3/1000
105/105 [==============================] - 0s 463us/step - loss: 0.7100 - accuracy: 0.6917
Epoch 4/1000
105/105 [==============================] - 0s 438us/step - loss: 0.6990 - accuracy: 0.6977
Epoch 5/1000
105/105 [==============================] - 0s 447us/step - loss: 0.6894 - accuracy: 0.6998
Epoch 6/1000
105/105 [==============================] - 0s 438us/step - loss: 0.6853 - accuracy: 0.6965
Epoch 7/1000
105/105 [==============================] - 0s 437us/step - loss: 0.6832 - accuracy: 0.7028
Epoch 8/1000
105/105 [==============================] - 0s 448us/step - loss: 0.6827 - accuracy: 0.6992
Epoch 9/1000
105/105 [==============================] - 0s 620us/step - loss: 0.6791 - accuracy: 0.6989
Epoch 10/1000
105/105 [==============================] - 0s 463u

## Feature Reduction

In [37]:
pca = PCA(n_components=0.99, whiten=True)

In [38]:
X_scaled_reduced = pca.fit_transform(X_scaled)

In [39]:
print(f'Original features: {X_scaled.shape[1]}')
print(f'Reduced features: {X_scaled_reduced.shape[1]}')

Original features: 10
Reduced features: 6


In [40]:
most_important = [np.abs(pca.components_[i]).argmax() for i in range(pca.components_.shape[0])]

In [41]:
initial_feature_names = features.columns

In [42]:
most_important_names = [initial_feature_names[most_important[i]] for i in range(pca.components_.shape[0])]

dic = {'PC{}'.format(i): most_important_names[i] for i in range(pca.components_.shape[0])}

In [43]:
# build the dataframe
df = pd.DataFrame(dic.items())

In [44]:
print(df)

     0             1
0  PC0  Whole Weight
1  PC1         Sex_M
2  PC2         Sex_I
3  PC3        Height
4  PC4        Length
5  PC5  Shell Weight


## Building Keras NN with PCA selected features best Tuning results

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled_reduced, target, random_state = 0, train_size = 0.8)

In [46]:
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.int32)

In [63]:
model = Sequential()

In [64]:
model.add(Dense(23, input_dim = X_train.shape[1], activation='relu'))
model.add(Dense(19, activation='relu'))
model.add(Dense(17, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [65]:
# must use categorical and sparse
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.Nadam(learning_rate=0.01),
              metrics=['accuracy'])

In [66]:
model.fit(X_train, y_train, epochs=1000)

Epoch 1/1000
105/105 [==============================] - 0s 603us/step - loss: 0.7851 - accuracy: 0.6726
Epoch 2/1000
105/105 [==============================] - 0s 482us/step - loss: 0.7236 - accuracy: 0.6830
Epoch 3/1000
105/105 [==============================] - 0s 455us/step - loss: 0.7080 - accuracy: 0.6890
Epoch 4/1000
105/105 [==============================] - 0s 485us/step - loss: 0.6997 - accuracy: 0.7001
Epoch 5/1000
105/105 [==============================] - 0s 472us/step - loss: 0.6928 - accuracy: 0.7040
Epoch 6/1000
105/105 [==============================] - 0s 484us/step - loss: 0.6955 - accuracy: 0.7016
Epoch 7/1000
105/105 [==============================] - 0s 468us/step - loss: 0.6823 - accuracy: 0.7052
Epoch 8/1000
105/105 [==============================] - 0s 476us/step - loss: 0.6854 - accuracy: 0.6983
Epoch 9/1000
105/105 [==============================] - 0s 470us/step - loss: 0.6804 - accuracy: 0.7103
Epoch 10/1000
105/105 [==============================] - 0s 471u

### Tuning: Adding more neurons

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(41, input_dim = X_train.shape[1], activation='relu'))
model.add(Dense(37, activation='relu'))
model.add(Dense(31, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [ ]:
# must use categorical and sparse
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.Nadam(learning_rate=0.01),
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=1000)